### Analyse search terms on the e-commerce web server


In [ ]:
# Install spark

In [ ]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [ ]:
# Start session

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Ecommerce Search Term Analysis") \
    .getOrCreate()
from pyspark.sql.functions import col
import urllib.request
import tarfile
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml.feature import VectorAssembler

25/06/15 20:35:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/15 20:35:51 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [ ]:
import urllib.request
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv"
urllib.request.urlretrieve(url, "searchterms.csv")

('searchterms.csv', <http.client.HTTPMessage at 0x74788633f290>)

In [ ]:
# Load the csv into a spark dataframe

In [ ]:
df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)

[Stage 1:>                                                          (0 + 1) / 1]

In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [ ]:
rowcount1 = df.count()
columncount1 = len(df.columns)
print(rowcount1)
print(columncount1)

10000
4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [ ]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [ ]:
print(df.schema["searchterm"].dataType)

StringType


In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [ ]:
gaming_laptop = df.filter(df.searchterm == "gaming laptop").count()
print(gaming_laptop)

499


In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [ ]:
top_5 = df.groupBy("searchterm").count().orderBy(col("count").desc()).limit(5).show()
print(top_5)
top_2 = df.groupBy("month").count().orderBy(col("count").desc()).limit(2).show()
print(top_2)

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+

None


[Stage 11:==================================================>  (190 + 10) / 200]

+-----+-----+
|month|count|
+-----+-----+
|   12| 5783|
|   11| 4217|
+-----+-----+

None


In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [ ]:
model_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz"
urllib.request.urlretrieve(model_url, "model.tar.gz")

with tarfile.open("model.tar.gz", "r:gz") as tar:
    tar.extractall(path="model")

model_path = "model/sales_prediction.model"
model = LinearRegressionModel.load(model_path)

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [ ]:
data_2023 = [(2023,)]
columns = ["year"]

df_2023 = spark.createDataFrame(data_2023, columns)

assembler = VectorAssembler(inputCols=["year"], outputCol="features")
df_2023_features = assembler.transform(df_2023).select("features")

predictions = model.transform(df_2023_features)
predictions.select("prediction").show()

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



25/06/15 20:36:32 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
25/06/15 20:36:32 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
